<a href="https://colab.research.google.com/github/laloflogar/RB2026/blob/main/INEGI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd # Import pandas for DataFrame and NaN

#Llamado al API
token='337db32f-2268-33f9-43e8-c853b84b6ee9'
serieID='910399'
url='https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/'+serieID+'/es/00/false/BIE-BISE/2.0/'+token+'?type=jsonn'
response= requests.get(url)
if response.status_code==200:
    content= json.loads(response.content)
    Series=content['Series'][0]['OBSERVATIONS']

    #Obtención de la lista de observaciones
    Observaciones=[]
    for obs in Series:
        # Check if 'OBS_VALUE' is None before attempting conversion
        if obs['OBS_VALUE'] is not None:
            Observaciones.append(float(obs['OBS_VALUE']))
        else:
            Observaciones.append(float('nan')) # Append NaN for None values


base = pd.DataFrame(Series, columns=['TIME_PERIOD', 'OBS_VALUE'])
# Ensure OBS_VALUE column in DataFrame is numeric, converting 'None' to NaN
base['OBS_VALUE'] = pd.to_numeric(base['OBS_VALUE'], errors='coerce')
base

,TIME_PERIOD,OBS_VALUE
0,2025/11,0.66
1,2025/10,0.36
2,2025/09,0.23
3,2025/08,0.06
4,2025/07,0.27
...,...,...
678,1969/05,0.00
679,1969/04,0.27
680,1969/03,0.10
681,1969/02,0.36


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd # Import pandas for datetime conversion

# Ensure 'TIME_PERIOD' is in datetime format
base['TIME_PERIOD'] = pd.to_datetime(base['TIME_PERIOD'], format='%Y/%m')

# Filter data for the years 2020 to 2025
filtered_base = base[(base['TIME_PERIOD'].dt.year >= 2020) & (base['TIME_PERIOD'].dt.year <= 2025)]

# Create a figure with secondary y-axis (though only one trace will be used here)
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add 'OBS_VALUE' as a line chart (red)
fig.add_trace(
    go.Scatter(
        x=filtered_base['TIME_PERIOD'],
        y=filtered_base['OBS_VALUE'],
        mode='lines',
        name='OBS_VALUE (2020-2025)',
        line=dict(color='red')
    ),
    secondary_y=False # Use primary y-axis
)

# Update layout to remove y-axis tick labels
fig.update_layout(
    title_text='OBS_VALUE over TIME_PERIOD (2020-2025)',
    yaxis=dict(
        title='Valor de Observación',
        showticklabels=False  # Remove y-axis tick values
    ),
    xaxis_title='Fecha'
)

fig.show()